# Simple MDP Model
$$
T \longleftrightarrow A \longleftrightarrow B \longleftrightarrow C \overset{1}{\longleftrightarrow} T
$$

Policy Evaluation (Prediction)  [See 4.1. Policy Evaluation, Sutton's book](http://incompleteideas.net/book/first/ebook/node41.html)

\begin{align}
    V^\pi & = \mathbb{E}_\pi \big\{   
                    r_{t+1} + \gamma r_{t+2} + ... | s_t = s
                \big\}  \\
          & =
             \mathbb{E}_\pi\big\{ r_{t+1} + \gamma V^\pi(s_{t+1})|s_t = s \big\} \\
          & =
              \sum_a \pi (s,a) \sum_{s'}  \mathcal{P}_{ss'}^a\big[ \mathcal{R}_{ss'}^a + \gamma V^\pi(s') \big]
\end{align}


## MC method for policy evaluation

In [47]:
import numpy as np

class Envornment:  # MDP
    
    def __init__(self, state_name_string='ABCDE'):
        rng = np.random.default_rng(2021)
        self.stat_names = list(state_name_string)
        self.stats = np.arange(len(self.stat_names))
        self.current = self.stats[rng.integers(0, len(self.stats))]
        
        self.reset()
        
    def step(self, action):  # action = [0, 1] left or right
        done = False
        if action == 0:
            self.current -= 1
            if self.current < 0:
                self.current = 0
                done = True
            reward = 0
            
        elif action == 1: # right
            self.current += 1
            if self.current >= len(self.stats):
                self.current -= 1
                done = True
                reward = 1
            else:
                reward = 0

        info = {}  # just to be gym
        
        return self.current, reward, done, info
    
    def reset(self,):
        self.current = self.stats[rng.integers(0, len(self.stats))]
        return self.current

In [48]:
# random action policy
rng = np.random.default_rng(2021)

def policy(state):
    return rng.integers(0, 2)

action_names = ['left', 'right']

In [80]:
env = Envornment('ABC')

state = env.reset()
print('init state: ', env.stat_names[state])

done = False
while done == False:
    action = policy(state)
    s, r, done, _ = env.step(action)
    print(action_names[action], s,r, done, end='  :  ')
    if done:    
        print(s, 'Terminated.', r, done)    
    else:
        print(s, env.stat_names[s], r, done)

init state:  B
right 2 0 False  :  2 C 0 False
left 1 0 False  :  1 B 0 False
left 0 0 False  :  0 A 0 False
left 0 0 True  :  0 Terminated. 0 True


In [107]:
def run_episode(env, verbose=True):
    sar = []
    state = env.reset()
    if verbose:
        print('init state: ', env.stat_names[state])

    done = False
    while done == False:
        action = policy(state)
        s, r, done, _ = env.step(action)
        
        sar.append((state, action, r))
        state = s  
        
        if verbose:
            print(action_names[action], s,r, done, end='  :  ')
            if done:    
                print(s, 'Terminated.', r, done)    
            else:
                print(s, env.stat_names[s], r, done)
    #
    return sar       

In [109]:
# First visit Monte-Carlo Value policy evaluation

env = Envornment('ABC')
N = np.zeros(len(env.stats))  # number of visit
V = np.zeros_like(N)          # value function

for _ in range(100):
    sar = run_episode(env, verbose=False)

    sar = np.array(sar)
    for i in range(sar.shape[0]):  # episode length
        start_state = sar[i, 0]
        G = sar[i:,-1].sum()  # total sum of rewards until termination
        N[start_state] += 1
        V[start_state] += G

V, N, V/N

(array([19., 67., 88.]),
 array([ 77., 114., 102.]),
 array([0.24675325, 0.5877193 , 0.8627451 ]))

In [112]:
# First visit Monte-Carlo Value policy evaluation
def policy_eval(niter=100):
    env = Envornment('ABC')
    N = np.zeros(len(env.stats))  # number of visit
    V = np.zeros_like(N)          # value function

    for _ in range(niter):
        sar = run_episode(env, verbose=False)

        sar = np.array(sar)
        for i in range(sar.shape[0]):  # episode length
            start_state = sar[i, 0]
            G = sar[i:,-1].sum()  # total sum of rewards until termination
            N[start_state] += 1
            V[start_state] += G
    return V/N

In [120]:
policy_eval(100)

array([0.35643564, 0.62318841, 0.80733945])

In [121]:
policy_eval(10000)

array([0.24053541, 0.49058718, 0.74494212])

In [122]:
policy_eval(100000)

array([0.24843128, 0.49856658, 0.74854524])

In [123]:
policy_eval(100000)

array([0.24983966, 0.5026176 , 0.75475681])

In [124]:
policy_eval(1000000)

array([0.2509998 , 0.50092254, 0.75066788])

In [125]:
policy_eval(1000000)

array([0.24910225, 0.49848645, 0.74886199])

In [126]:
policy_eval(10000000)

array([0.25019204, 0.5002766 , 0.75026337])

## Exact solution to the policy evalution problem of the MDP Model
$$
T \longleftrightarrow A \longleftrightarrow B \longleftrightarrow C \overset{1}{\longleftrightarrow} T
$$

Policy Evaluation (Prediction)  [See 4.1. Policy Evaluation, Sutton's book](http://incompleteideas.net/book/first/ebook/node41.html)

\begin{align}
    V^\pi & = \mathbb{E}_\pi \big\{   
                    r_{t+1} + \gamma r_{t+2} + ... | s_t = s
                \big\}  \\
          & =
             \mathbb{E}_\pi\big\{ r_{t+1} + \gamma V^\pi(s_{t+1})|s_t = s \big\} \\
          & =
              \sum_a \pi (s,a) \sum_{s'}  \mathcal{P}_{ss'}^a\big[ \mathcal{R}_{ss'}^a + \gamma V^\pi(s') \big]
\end{align}


With random action policy,

V(A) = A = 1/2 (0 + 0 + B)
V(B) = B = 1/2 (0 + A + 0 + C)
V(C) = C = 1/2 (0 + B + 1)

V = [ [0, .5, 0], [.5, 0, .5], [0, .5, 0] ] V + [0, 0, .5]

V = inv(I - M) [0, 0, .5]

In [133]:
I = np.eye(3); I

array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.]])

In [134]:
A = np.array([[0, .5, 0], [.5, 0, .5], [0, .5, 0]])

b = np.array([0,0,.5])

In [137]:
np.linalg.inv(I-A)

array([[1.5, 1. , 0.5],
       [1. , 2. , 1. ],
       [0.5, 1. , 1.5]])

In [138]:
np.linalg.inv(I-A) @ b

array([0.25, 0.5 , 0.75])

## Exact solution to the policy evalution problem
$$
T \longleftrightarrow A \longleftrightarrow \ldots \overset{0}{\longleftrightarrow} E \overset{1}{\longleftrightarrow} T
$$

Policy Evaluation (Prediction)  [See 4.1. Policy Evaluation, Sutton's book](http://incompleteideas.net/book/first/ebook/node41.html)

\begin{align}
    V^\pi & = \mathbb{E}_\pi \big\{   
                    r_{t+1} + \gamma r_{t+2} + ... | s_t = s
                \big\}  \\
          & =
             \mathbb{E}_\pi\big\{ r_{t+1} + \gamma V^\pi(s_{t+1})|s_t = s \big\} \\
          & =
              \sum_a \pi (s,a) \sum_{s'}  \mathcal{P}_{ss'}^a\big[ \mathcal{R}_{ss'}^a + \gamma V^\pi(s') \big]
\end{align}


In [139]:
I = np.eye(5)
A = np.array([ [0, .5, 0, 0, 0],
               [.5, 0, .5, 0, 0],
               [0, .5, 0, .5, 0],
               [0, 0, .5, 0, .5],
               [0, 0, 0, .5, 0]])
b = np.array([0, 0, 0, 0, .5])

In [140]:
np.linalg.inv(I - A) @ b

array([0.16666667, 0.33333333, 0.5       , 0.66666667, 0.83333333])

End.